In [7]:
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score

In [17]:
test_dataset = dataset["test"].shuffle(seed=42).select(range(500))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

test_dataset = test_dataset.map(tokenize_function, batched=True)

test_dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])

In [21]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.eval()
all_preds = []
all_labels = []

for batch in torch.utils.data.DataLoader(test_dataset, batch_size=8):
    with torch.no_grad():
        # attention_mask 인자를 올바르게 전달
        output = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
    
    logits = output.logits
    preds = np.argmax(logits.numpy(), axis=1)
    
    # 라벨을 numpy 배열로 변환
    all_preds.extend(preds)
    all_labels.extend(batch['label'].numpy())

# 정확도 계산
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy without fine-tuning: {accuracy:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy without fine-tuning: 0.5080
